In [5]:
import sys
import cPickle
import math
import json

import uproot
import numpy
import root_numpy
from awkward import *

Welcome to JupyROOT 6.15/01


In [9]:
%%timeit
q = root_numpy.root2array("../NanoAOD-DYJetsToLL.root", "tree", ["Muon_pt", "Muon_eta", "Muon_phi"])

1 loop, best of 3: 770 ms per loop


In [2]:
tree = uproot.open("../NanoAOD-DYJetsToLL.root")["tree"]

In [23]:
tree.keys(filtername=lambda x: "MET" in x)

['CaloMET_phi',
 'CaloMET_pt',
 'CaloMET_sumEt',
 'GenMET_phi',
 'GenMET_pt',
 'MET_MetUnclustEnUpDeltaX',
 'MET_MetUnclustEnUpDeltaY',
 'MET_covXX',
 'MET_covXY',
 'MET_covYY',
 'MET_fiducialGenPhi',
 'MET_fiducialGenPt',
 'MET_phi',
 'MET_pt',
 'MET_significance',
 'MET_sumEt',
 'PuppiMET_phi',
 'PuppiMET_pt',
 'PuppiMET_sumEt',
 'RawMET_phi',
 'RawMET_pt',
 'RawMET_sumEt',
 'TkMET_phi',
 'TkMET_pt',
 'TkMET_sumEt',
 'HLT_CaloMHTNoPU90_PFMET90_PFMHT90_IDTight',
 'HLT_CaloMHTNoPU90_PFMET90_PFMHT90_IDTight_BTagCSV_p067',
 'HLT_DiPFJet40_DEta3p5_MJJ600_PFMETNoMu140',
 'HLT_DiPFJet40_DEta3p5_MJJ600_PFMETNoMu80',
 'HLT_DoubleMu3_PFMET50',
 'HLT_Ele15_IsoVVVL_PFHT350_PFMET50',
 'HLT_Ele15_IsoVVVL_PFHT400_PFMET50',
 'HLT_HT250_CaloMET70',
 'HLT_IsoMu16_eta2p1_MET30',
 'HLT_IsoMu16_eta2p1_MET30_LooseIsoPFTau50_Trk30_eta2p1',
 'HLT_LooseIsoPFTau50_Trk30_eta2p1_MET110',
 'HLT_LooseIsoPFTau50_Trk30_eta2p1_MET120',
 'HLT_LooseIsoPFTau50_Trk30_eta2p1_MET80',
 'HLT_LooseIsoPFTau50_Trk30_eta2p1_MET

In [22]:
tree["nMuon"].interpretation

asdtype('>u4')

In [4]:
%%timeit
pt, eta, phi = tree.arrays(["Muon_pt", "Muon_eta", "Muon_phi"], outputtype=tuple)

10 loops, best of 3: 46.9 ms per loop


In [3]:
pt, eta, phi = tree.arrays(["Muon_pt", "Muon_eta", "Muon_phi"], outputtype=tuple)

In [4]:
pt.content.dtype, eta.content.dtype, phi.content.dtype

(dtype('float32'), dtype('float32'), dtype('float32'))

In [5]:
len(pt), len(eta), len(phi)

(701716, 701716, 701716)

In [6]:
len(pt.content), len(eta.content), len(phi.content)

(552056, 552056, 552056)

In [24]:
events = JaggedArray.fromoffsets(pt.offsets, Table(len(pt.content), pt=pt.content, eta=eta.content, phi=phi.content))

In [25]:
((sys.getsizeof(events._content._content["pt"]) + events._content._content["pt"].nbytes +
  sys.getsizeof(events._content._content["eta"]) + events._content._content["eta"].nbytes +
  sys.getsizeof(events._content._content["phi"]) + events._content._content["phi"].nbytes) +
 sys.getsizeof("pt") + sys.getsizeof("eta") + sys.getsizeof("phi") + sys.getsizeof(events._content._content) +
 sys.getsizeof(events._content._length) + sys.getsizeof(events._content._step) + sys.getsizeof(events._content._start) +
 sys.getsizeof(events._content) + sys.getsizeof(events._writeable) +
 sys.getsizeof(events._starts) + sys.getsizeof(events._stops) + events.offsets.nbytes + sys.getsizeof(events))

12239527

In [26]:
(4*len(pt.content) + 4*len(eta.content) + 4*len(phi.content) + 24*len(pt) + 24)

23465880

In [27]:
%%timeit
events.content["pz"] = (events["pt"] * numpy.sinh(events["eta"])).content

10 loops, best of 3: 30.1 ms per loop


In [28]:
events[:6].tolist()

[[],
 [{'eta': -1.005615234375,
   'phi': -0.5814208984375,
   'pt': 129.7563018798828,
   'pz': -153.61647033691406},
  {'eta': -0.7191162109375,
   'phi': -1.5068359375,
   'pt': 73.08880615234375,
   'pz': -57.20792007446289}],
 [{'eta': -0.177520751953125,
   'phi': 0.46759033203125,
   'pt': 127.55380249023438,
   'pz': -22.762563705444336},
  {'eta': -0.697509765625,
   'phi': 1.11865234375,
   'pt': 93.02200317382812,
   'pz': -70.2744369506836}],
 [],
 [],
 [{'eta': 1.1064453125,
   'phi': 0.7440185546875,
   'pt': 117.4339828491211,
   'pz': 158.11656188964844},
  {'eta': 2.06201171875,
   'phi': 0.03387451171875,
   'pt': 46.91141128540039,
   'pz': 181.4198455810547}]]

In [29]:
%%timeit
pz = events["pt"] * numpy.sinh(events["eta"])

10 loops, best of 3: 30.3 ms per loop


In [30]:
events[1]["pt"][0] * numpy.sinh(events[1]["eta"][0])

-153.61647

In [32]:
(pt.stops - pt.starts).max()

8

In [63]:
class Muon(object):
    __slots__ = ["pt", "eta", "phi"]
    def __init__(self, pt, eta, phi):
        self.pt, self.eta, self.phi = pt, eta, phi

asdicts = []
asobjs = []
for event in events:
    dictevt = []
    objevt = []
    for muon in event:
        dictevt.append({"pt": float(muon.pt), "eta": float(muon.eta), "phi": float(muon.phi)})
        objevt.append(Muon(float(muon.pt), float(muon.eta), float(muon.phi)))
    asdicts.append(dictevt)
    asobjs.append(objevt)

In [64]:
len(asdicts), len(asobjs)

(701716, 701716)

In [65]:
asdicts_serialized = cPickle.dumps(asdicts, protocol=2)
asobjs_serialized = cPickle.dumps(asobjs, protocol=2)

In [66]:
type(asdicts_serialized), type(asobjs_serialized)

(str, str)

In [67]:
len(asdicts_serialized), len(asobjs_serialized)

(27373409, 33998099)

In [77]:
asdicts_json = json.dumps(asdicts, separators=(',', ':'))
type(asdicts_json), len(asdicts_json), asdicts_json[:100]

(str,
 38992393,
 '[[],[{"phi":-0.5814208984375,"eta":-1.005615234375,"pt":129.7563018798828},{"phi":-1.5068359375,"eta')

In [78]:
asdicts_json = json.dumps(json.loads(json.dumps(asdicts), parse_float=lambda x: round(float(x), 1)), separators=(',', ':'))
type(asdicts_json), len(asdicts_json), asdicts_json[:100]

(str,
 20122887,
 '[[],[{"phi":-0.6,"eta":-1.0,"pt":129.8},{"phi":-1.5,"eta":-0.7,"pt":73.1}],[{"phi":0.5,"eta":-0.2,"p')

In [45]:
pz = numpy.empty(552056, dtype=numpy.float32)

In [52]:
%%timeit
i = 0
for event in asdicts:
    for muon in event:
        pz[i] = muon["pt"] * math.sinh(muon["eta"])
        i += 1

1 loop, best of 3: 1.23 s per loop


In [54]:
%%timeit
i = 0
for event in asobjs:
    for muon in event:
        pz[i] = muon.pt * math.sinh(muon.eta)
        i += 1

1 loop, best of 3: 1.24 s per loop


In [86]:
totbytes = sys.getsizeof(asdicts)
for event in asdicts:
    totbytes += sys.getsizeof(event)
    for muon in event:
        totbytes += sys.getsizeof(muon)
        for n, x in muon.items():
            totbytes += sys.getsizeof(n)
            totbytes += sys.getsizeof(x)
totbytes

327099392

In [100]:
totbytes = sys.getsizeof(asdicts)
for event in asobjs:
    totbytes += sys.getsizeof(event)
    for muon in event:
        totbytes += sys.getsizeof(muon)
        totbytes += sys.getsizeof(muon.pt)
        totbytes += sys.getsizeof(muon.eta)
        totbytes += sys.getsizeof(muon.phi)
totbytes

146577080

In [101]:
import numba

In [102]:
@numba.njit
def doit(pz, starts, stops, pt, eta):
    k = 0
    for i in range(len(starts)):
        for j in range(starts[i], stops[i]):
            pz[k] = pt[j] * math.sinh(eta[j])
            k += 1

In [104]:
%%timeit
doit(pz, pt.starts, pt.stops, pt.content, eta.content)

10 loops, best of 3: 22.6 ms per loop
